<a href="https://colab.research.google.com/github/marcosandrey85/BandasBollinger/blob/main/Bandas_de_Bollinger.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importando as bibliotecas

In [1]:
!pip install yfinance 
import yfinance as yf
import pandas as pd
import numpy as np


     |████████████████████████████████| 6.4 MB 13.1 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


# Ação que queremos trabalhar

In [2]:
acao = yf.Ticker('OIBR3.SA')
data = acao.history(period ="1y" )
df = data[['Close']]
df

,Close
Date,
2021-02-25,1.86
2021-02-26,1.88
2021-03-01,1.84
2021-03-02,1.79
2021-03-03,1.76
...,...
2022-02-21,0.81
2022-02-22,0.81
2022-02-23,0.80


# Calculos estatiticos 

### Média móvel

In [3]:
mm = df.rolling(window = 20).mean()
mm

,Close
Date,
2021-02-25,NaN
2021-02-26,NaN
2021-03-01,NaN
2021-03-02,NaN
2021-03-03,NaN
...,...
2022-02-21,0.9660
2022-02-22,0.9620
2022-02-23,0.9565


### Desvio padrão

In [4]:
dpm = df.rolling(window=20).std()
dpm

,Close
Date,
2021-02-25,NaN
2021-02-26,NaN
2021-03-01,NaN
2021-03-02,NaN
2021-03-03,NaN
...,...
2022-02-21,0.079697
2022-02-22,0.085508
2022-02-23,0.092297


### Calculo da Banda superior e inferior

In [13]:
sup_band = mm + (2*dpm)
inf_band = mm - (2*dpm)

**Alterando o nome das colunas de banda sup e banda inf**

In [14]:
sup_band= sup_band.rename(columns = {'Close' : 'superior'})
inf_band= inf_band.rename(columns = {'Close' : 'inferior'})

# Unir todas as colunas

In [15]:
bandas_bollinger = df.join(sup_band).join(inf_band)
bandas_bollinger

,Close,superior,inferior
Date,,,
2021-02-25,1.86,NaN,NaN
2021-02-26,1.88,NaN,NaN
2021-03-01,1.84,NaN,NaN
2021-03-02,1.79,NaN,NaN
2021-03-03,1.76,NaN,NaN
...,...,...,...
2022-02-21,0.81,1.125394,0.806606
2022-02-22,0.81,1.133016,0.790984
2022-02-23,0.80,1.141093,0.771907


### Retirar os nulos

In [16]:
bandas_bollinger = bandas_bollinger.dropna()


# Calculo dos pontos de compra e venda

In [17]:
compra = bandas_bollinger[bandas_bollinger['Close'] <= bandas_bollinger['inferior']]
venda =  bandas_bollinger[bandas_bollinger['Close'] >= bandas_bollinger['superior']]

# Plotando o Gráfico

In [18]:
import plotly.io as pio 
import plotly.graph_objects as go

pio.templates.default = "plotly_dark"

fig = go.Figure()
fig.add_trace(go.Scatter(x = inf_band.index,
                         y = inf_band['inferior'],
                         name = 'Banda Inferior',
                         line_color = 'rgba(173,204,255,0.2)'
                         ))
fig.add_trace(go.Scatter(x = sup_band.index,
                         y = sup_band['superior'],
                         name = 'Banda Superior',
                         fill = 'tonexty',
                         fillcolor = 'rgba(173,204,255,0.2)',
                         line_color = 'rgba(173,204,255,0.2)'
                         ))
fig.add_trace(go.Scatter(x = df.index,
                         y = df['Close'],
                         name = 'Preço fechamento',
                         line_color = '#636EFA'
                         ))
fig.add_trace(go.Scatter(x = mm.index,
                         y = mm['Close'],
                         name = 'Média móvel',
                         line_color = '#FECB52'
                         ))
fig.add_trace(go.Scatter(x = compra.index,
                         y = compra['Close'],
                         name = 'Compra',
                         mode = 'markers',
                         marker = dict(
                             color = '#00CC96',
                             size = 8
                             )
                         ))
fig.add_trace(go.Scatter(x = venda.index,
                         y = venda['Close'],
                         name = 'Venda',
                         mode = 'markers',
                         marker = dict(
                             color = '#EF553B',
                             size = 8
                             )
                        ))
fig.show()

In [11]:
bandas_bollinger

,Close,superior,inferior
Date,,,
2021-03-24,1.80,1.843965,1.835035
2021-03-25,1.80,1.840992,1.832008
2021-03-26,1.80,1.836967,1.828033
2021-03-29,1.89,1.839556,1.830444
2021-03-30,1.92,1.846178,1.836822
...,...,...,...
2022-02-21,0.81,0.971579,0.960421
2022-02-22,0.81,0.967986,0.956014
2022-02-23,0.80,0.962961,0.950039


In [12]:
import plotly
